In [1]:
import math

import numpy
import numpy as np
import scipy


def black_scholes(spot_price, strike_price, risk_rate, mature_time, volitility, callput):
    d1 = (math.log(spot_price / strike_price) + (risk_rate + ((volitility ** 2) / 2)) * mature_time) / (
            volitility * math.sqrt(mature_time))
    d2 = (d1 - (volitility * math.sqrt(mature_time)))
    call = (spot_price * scipy.stats.norm.cdf(d1)) - strike_price * math.exp(
        -risk_rate * mature_time) * scipy.stats.norm.cdf(d2)
    put = strike_price * math.exp(-risk_rate * mature_time) * scipy.stats.norm.cdf(
        -d2) - spot_price * scipy.stats.norm.cdf(-d1)
    if callput == "call":
        return call
    else:
        return put
    # call = (spot_price * scipy.stats.norm.cdf((math.log(spot_price/strike_price)+(risk_rate+((volitility**2)/2))*mature_time) / (volitility*math.sqrt(mature_time)))) - strike_price * math.exp(-risk_rate * mature_time) * scipy.stats.norm.cdf(((math.log(spot_price/strike_price)+(risk_rate+((volitility**2)/2))*mature_time) / (volitility*math.sqrt(mature_time)) - (volitility * math.sqrt(mature_time))))

In [5]:
print(black_scholes(100, 120, 0.05, 0.5, 0.2, "call"))
print(black_scholes(100, 120, 0.05, 0.5, 0.2, "put"))
print(black_scholes(100,94,0.05, 0.5, 0.2, "call"))
print(black_scholes(100,94,0.05, 0.5, 0.2, "put"))
# black_scholes(spot_price,strike_price,risk_rate,mature_time,volatility)

1.0226152225558796
18.05980466595578
10.549187590491826
2.2283193211550945


In [31]:
import math
import numpy
import scipy


def binomial(starting_value, strike, rate, volitility, time, step, type, excersise):
    up = math.exp(volitility * math.sqrt(time / step))
    down = 1 / math.exp(volitility * math.sqrt(time / step))
    rate_value = math.exp((-rate) * time / step)
    increase_probability = ((math.exp(rate * (time / step)) - down) / (up - down))
    decrease_probability = 1 - ((math.exp(rate * (time / step)) - down) / (up - down))
    tree = [[0]]
    tree[0][0] = tree[0][0] + starting_value
    for cur_depth in range(1, step + 1):
        new_values = []
        old = tree[-1]
        for child in range(0, cur_depth + 1):
            if cur_depth == child:
                new = old[-1] * down
            elif child == 0:
                new = old[0] * up
            else:
                new = ((old[child - 1] * up) + (old[child] * down)) / 2
            new_values.append(new)
        tree.append(new_values)
    children = tree[-1]
    values = []
    for i in children:
        if type == "put":
            if strike - i > 0:
                option = strike - i
            else:
                option = 0
        else:
            if i - strike > 0:
                option = i - strike
            else:
                option = 0
        values.append(option)
    i = step - 1
    while i >= 0:
        for j in range(i + 1):
            values[j] = (increase_probability * values[j + 1] + decrease_probability * values[j]) * rate_value
        i = i - 1
    # print(tree)
    print(values)
    return values[0]


# print(binomial(100, 120, 0.05, 0.2, 0.5,183,"put","e"))
#binomial(100,110,0.05,0.2,1,100,'call','euro')
# binomial(20,21,0.12,0.1,0.5,2,"call","euro")
binomial(50, 52, 0.05, 0.2, 2, 2, "put", "e")

[5.577764856335021, 10.153787114456259, 18.483997698218033]


5.577764856335021

In [49]:
import numpy


def monte_carlo(stock_price,strike_price,time, interest_rate, volitility, type,steps,simulations ):
    dt = time/steps
    rn = (interest_rate-0.5*volitility**2)*dt
    vol = volitility*numpy.sqrt(dt)
    z = numpy.random.normal(size=(steps,simulations))
    st = stock_price*numpy.exp(numpy.cumsum(rn+vol*z,axis=0))
    if type=="call":
        values=numpy.maximum(st[-1]-strike_price,0)
    else:
        values=numpy.maximum(strike_price-st[-1],0)
    return  numpy.mean(values)*numpy.exp(-interest_rate*time)




print(monte_carlo(100,120, 0.5, 0.05,0.2,"call", 252,100000))
print(monte_carlo(100,120, 0.5, 0.05,0.2,"put", 252,100000))
print(monte_carlo(100,94, 0.5, 0.05,0.2,"call", 252,100000))
print(monte_carlo(100,94, 0.5, 0.05,0.2,"put", 252,100000))

1.0403885223004867
18.077924434054655
10.516177934207803
2.235947303999189
